In [ ]:
!gdown --id 1CWCo7GFk3lZky1EXjZ6XpHZS5tUbBOj1

In [ ]:
from sklearn.preprocessing import QuantileTransformer
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from sklearn.metrics import mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors


In [ ]:
data_all = pd.read_csv('/content/Features_no_filtering_ordered.csv')

In [ ]:
models = ['KNN Regressor']
metrics = ['MAPE', 'MAE', 'R2']
formes = ['inverted', 'average']
columns = [
    np.array([models[i] for i in range(1) for j in range(3) for k in range(2)]),
    np.array([metrics[j] for i in range(1) for j in range(3) for k in range(2)]),
    np.array([formes[k] for i in range(1) for j in range(3) for k in range(2)]),
]
chans = ['Fz', 'F3', 'F7','C3', 'T7', 'Pz', 'P3','P7', 'O1', 'Oz', 'O2','P4', 'P8', 'Cz','C4', 'T8', 'F4', 'F8']

In [ ]:
import warnings
import logging
warnings.filterwarnings('ignore')

logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)
logging.getLogger("np").setLevel(logging.WARNING)

In [ ]:
result = pd.DataFrame()
grid_res = []
n_films = 8
n_subjects = 32
result = pd.DataFrame()
for film in tqdm_notebook(range(1, n_films + 1)):
    df = pd.DataFrame(np.random.randn(1, 1 * 3 * 2), columns=columns)
    X_test = data_all.loc[(data_all['film'] == film)].reset_index(drop=True)
    X_train = data_all.loc[(data_all['film'] != film)].reset_index(drop=True)
    X_train = X_train.sample(frac=1)
    X_test = X_test.sample(frac=1)

    y_train = X_train['labels']
    y_test_inv = np.array(X_test['labels'])
    train_film = X_train['film']
    train_subj = X_train['Subj']
    train_ch = X_train['ch']
    test_film = X_test['film']
    test_subj = X_test['Subj']
    test_ch = np.array(X_test['ch'])
    
    X_train = X_train.drop(['film', 'Subj', 'labels'], axis=1)
    X_test = X_test.drop(['film', 'Subj', 'labels'], axis=1)
    
    qt_train = QuantileTransformer(output_distribution='normal',
                             copy=True)
    y_train = pd.DataFrame(qt_train.fit_transform(np.array(y_train).reshape(-1, 1)))
    qt_test = QuantileTransformer(output_distribution='normal',
                             copy=True)
    y_test = pd.DataFrame(qt_test.fit_transform(np.array(y_test_inv).reshape(-1, 1)))

    n_neighbors = [100, 300, 500, 1000]
    param_grid = dict(n_neighbors=n_neighbors)
    grid = GridSearchCV(estimator=neighbors.KNeighborsRegressor(), param_grid=param_grid, cv=[(slice(None), slice(None))])
    grid_result = grid.fit(X_train, y_train)
    model = neighbors.KNeighborsRegressor(n_neighbors=grid.best_params_['n_neighbors']).fit(X_train, y_train)
    y_pred = model.predict(X_test)
    preds = qt_test.inverse_transform(np.array(y_pred).reshape(-1, 1))
    df[('KNeighborsRegressor', 'R2', 'inverted')] = r2_score(preds, y_test_inv)
    df[('KNeighborsRegressor', 'MAPE', 'inverted')] = mean_absolute_percentage_error(preds, y_test_inv)
    df[('KNeighborsRegressor', 'MAE', 'inverted')] = mean_absolute_error(preds, y_test_inv)
    
    pred = [[0 for j in range(n_films)] for i in range(n_subjects)]
    num_chans = [[0 for j in range(n_films)] for i in range(n_subjects)]
    labels = [[0 for j in range(n_films)] for i in range(n_subjects)]
    lbl = [[0 for j in range(n_films)] for i in range(n_subjects)]
    channels = [[] for i in range(n_subjects)]
    for i in range(len(preds)):
        pred[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += preds[i]
        num_chans[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += 1
        labels[np.array(test_subj)[i] - 1][np.array(test_film)[i] - 1] += y_test_inv[i]
        channels[np.array(test_subj)[i] - 1].append(chans[test_ch[i] - 1])
    labels = np.array(labels).astype(np.float64) / np.array(num_chans).astype(np.float64)
    pred = np.array(pred).astype(np.float64) / np.array(num_chans).astype(np.float64)
    labels = np.nan_to_num(labels)
    pred = np.nan_to_num(pred)
    predicted = []
    lbl = []
    for i in range(len(labels)):
        if sum(labels[i]) != 0:
            for j in range(n_films):
                if labels[i][j] != 0:
                    lbl.append(labels[i][j])  
                    predicted.append(pred[i][j])
    df[('KNeighborsRegressor', 'R2', 'average')] = r2_score(lbl, predicted)
    df[('KNeighborsRegressor', 'MAPE', 'average')] = mean_absolute_percentage_error(lbl, predicted)
    df[('KNeighborsRegressor', 'MAE', 'average')] = mean_absolute_error(lbl, predicted)
    result = pd.concat([result, df], axis=0)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
result.describe()

KNN Regressor                      ... KNeighborsRegressor                    
               MAPE                 MAE  ...                  R2      MAPE       MAE
           inverted   average  inverted  ...             average   average   average
count      8.000000  8.000000  8.000000  ...            8.000000  8.000000  8.000000
mean      -0.094073  0.352425  0.348748  ...            0.018854  0.361571  1.297218
std        1.299048  0.626360  0.942948  ...            0.063022  0.162119  0.255866
min       -2.199379 -0.424508 -1.590056  ...           -0.059267  0.143680  0.854581
25%       -0.910564 -0.012428  0.022469  ...           -0.010325  0.267591  1.168784
50%        0.220442  0.185172  0.499327  ...           -0.006724  0.309609  1.278953
75%        0.631355  0.561038  0.820882  ...            0.042294  0.455214  1.438398
max        1.579229  1.540457  1.476902  ...            0.143143  0.642493  1.638809

[8 rows x 12 columns]